In [1]:
import numpy as np
import scipy as sp
from utility import M,F, benchmark_mapping, get_best_connected, invert, score, get_highest_out_degrees

In [5]:
def find_best_move(G,H, mapping, u, mask = None):

    m = G.shape[0]
    u_row = G[u,:]
    u_col = G[:,u]

    if mask is None:
        n = H.shape[0]
        X = H[:,mapping[None,:]].reshape((n,m))
        Y = H[mapping[:,None],:].reshape((m,n))
    else:
        X = H[mask,mapping[None,:]]
        n = X.shape[0]
        X = X.reshape((n,m))
        Y = H[mapping[:,None],mask].reshape((m,n))

    scores = np.minimum(u_row[None,:],X).sum(axis=1) + np.minimum(u_col[:,None],Y).sum(axis=0)
    scores -= np.minimum(G[u,u],H.diagonal())

    return scores

In [36]:
m,n = 50,500
H_M,m_labels = get_highest_out_degrees(M,n)
H_F,f_labels = get_highest_out_degrees(F,n)
H_M = H_M.todense()
H_F = H_F.todense()
H_F[np.arange(n),np.arange(n)] = 0
H_M[np.arange(n),np.arange(n)] = 0
np.count_nonzero(H_M)/n**2, np.count_nonzero(H_F)/n**2

(0.204576, 0.112264)

In [78]:
rng = np.random.default_rng()
result_matrix = np.zeros((n,n),dtype=int)
score_track = []
mapping_save = []

for k in range(500):

    mapping_M = rng.choice(n,m,replace=False)
    mapping_F = rng.choice(n,m,replace=False)

    for i in range(50):

        outer_unchanged = True

        if i % 2 == 0:
            G = H_F[mapping_F[:,None],mapping_F[None,:]]
            H = H_M
            mapping = mapping_M.copy()
        else:
            G = H_M[mapping_M[:,None],mapping_M[None,:]]
            H = H_F
            mapping = mapping_F.copy()

        unmapped = np.ones(n,dtype=bool)
        unmapped[mapping] = False
        top_score = score(G,H,mapping,require_surjective=False)
        
        for j in range(50):

            unchanged = True

            for u in rng.permutation(m):

                change_scores = find_best_move(G,H,mapping,u)
                v = (change_scores*unmapped).argmax()

                if change_scores[v] > change_scores[mapping[u]]:
                    top_score = top_score + change_scores[v]-change_scores[mapping[u]]

                    unmapped[mapping[u]] = True
                    unmapped[v] = False
                    mapping[u] = v
                    unchanged = False
                    outer_unchanged = False

            if unchanged:
                break
        
        if i % 2 == 0:
            mapping_M = mapping.copy()
        else:
            mapping_F = mapping.copy()

        if outer_unchanged:
            print(f"{k=}, {i=}, {top_score=}"+" "*20,end="\r")
            break

    A = H_F[mapping_F[:,None],mapping_F[None,:]]
    B = H_M[mapping_M[:,None],mapping_M[None,:]]

    score_track.append((score(G,H,mapping,require_surjective=False), score(A,A,np.arange(m)), score(B,B,np.arange(m))))
    mapping_save.append((mapping_F,mapping_M))
    result_matrix[mapping_F,mapping_M] += 1

In [125]:
track = np.array(score_track)
track[:,0].argmax()
track[13]

array([ 9963, 22444, 21916], dtype=int32)

In [109]:
mapping_F,mapping_M=mapping_save[403]

In [95]:
np.unravel_index(result_matrix.argmax(),result_matrix.shape)
m_labels[1],f_labels[27]
sorted = np.sort(result_matrix.flatten())
sorted[-50:]

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,
       12, 12, 13, 13, 13, 14, 14, 15, 15, 15, 15, 15, 15, 15, 17, 20])

In [ ]:
np.concatenate([m_labels[mapping_M,None],f_labels[mapping_F,None]],axis=1)

In [106]:
A=M.todense()
B=F.todense()
N=A.shape[0]

In [ ]:
from algorithms import greedy_mapping
start_mapping=-np.ones(N,dtype=int)
inverse_benchmark = invert(benchmark_mapping)
#start_mapping[f_labels[mapping_F]]=m_labels[mapping_M]
random_set = rng.choice(N,500,replace=False)
start_mapping[random_set] = inverse_benchmark[random_set]
mapping = greedy_mapping(B,A,start_mapping)

In [127]:
score(F,M,mapping),score(F,M,invert(benchmark_mapping))

(1382035, 5154247)

In [123]:
np.save("Mapping-11-12-24 (5.18Mio).arr",mapping)